# A1 - MLP for image and text classification

- Pytorch MLP image classification CIFAR10
- Keras MLP image classification CIFAR10
- Pytorch MLP text classification IMDB Movie reviews
- Keras MLP text classification IMDB Movie reviews

## Pytorch MLP image classification CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim

### Import and Preprocess the data

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


100%|██████████| 170M/170M [00:03<00:00, 49.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=False, num_workers=2)


In [5]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


#### Class definition

In [6]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Defining the model architecture Using Pytorch

Use of cuda to put our data and model on GPU

In [7]:
device=torch.device('cuda:0')

MLP Structure

In [8]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = torch.nn.Linear(32 * 32 * 3,64)
        self.fc2 = torch.nn.Linear(64,64)
        self.fc3 = torch.nn.Linear(64,64)
        self.fc4 = torch.nn.Linear(64,10)

    def forward(self,x):
        x = x.view(-1,32 * 32 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [9]:
mlp = MLP().to(device)
summary(mlp,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]         196,672
            Linear-2                   [-1, 64]           4,160
            Linear-3                   [-1, 64]           4,160
            Linear-4                   [-1, 10]             650
Total params: 205,642
Trainable params: 205,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.78
Estimated Total Size (MB): 0.80
----------------------------------------------------------------


### Model training

Before training:
- Loss function: to measure how good the network is. We choose Cross entropy loss

- Optimizer: to update network as it sees more data and reduce loss value. We use SGD optimizer.

- Metrics: to monitor performance of network. We evaluate the model by using accuracy.

In [10]:
criterion = nn.CrossEntropyLoss()

Functions of train and test:

In [11]:
def train_cuda(running_loss,net,optimizer,epoch):
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if i % 20 == 19:
            print("Epoch:{}, Iteration:{}, Loss:{:.3f}".format(epoch + 1, i + 1, running_loss / (i+1)))


In [12]:
def test_cuda(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Training

In [13]:
optimizer_mlp = optim.SGD(mlp.parameters(), lr=0.05, momentum=0.9)
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    train_cuda(running_loss,mlp,optimizer_mlp,epoch)
    test_cuda(mlp)

print('Finished Training')

Epoch:1, Iteration:20, Loss:2.302
Epoch:1, Iteration:40, Loss:2.302
Epoch:1, Iteration:60, Loss:2.301
Epoch:1, Iteration:80, Loss:2.300
Epoch:1, Iteration:100, Loss:2.298
Epoch:1, Iteration:120, Loss:2.293
Epoch:1, Iteration:140, Loss:2.289
Epoch:1, Iteration:160, Loss:2.283
Epoch:1, Iteration:180, Loss:2.277
Epoch:1, Iteration:200, Loss:2.271
Epoch:1, Iteration:220, Loss:2.266
Epoch:1, Iteration:240, Loss:2.262
Epoch:1, Iteration:260, Loss:2.257
Epoch:1, Iteration:280, Loss:2.252
Epoch:1, Iteration:300, Loss:2.247
Epoch:1, Iteration:320, Loss:2.243
Epoch:1, Iteration:340, Loss:2.239
Epoch:1, Iteration:360, Loss:2.235
Epoch:1, Iteration:380, Loss:2.231
Accuracy of the network on the test images: 31 %
Epoch:2, Iteration:20, Loss:2.133
Epoch:2, Iteration:40, Loss:2.132
Epoch:2, Iteration:60, Loss:2.127
Epoch:2, Iteration:80, Loss:2.124
Epoch:2, Iteration:100, Loss:2.121
Epoch:2, Iteration:120, Loss:2.118
Epoch:2, Iteration:140, Loss:2.118
Epoch:2, Iteration:160, Loss:2.116
Epoch:2, Itera

## Keras MLP image classification CIFAR10

In [14]:
import numpy as np


from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.optimizers import SGD

In [15]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [16]:
print("Shape of training data:")
print(X_train.shape)
print(y_train.shape)
print("Shape of test data:")
print(X_test.shape)
print(y_test.shape)

Shape of training data:
(50000, 32, 32, 3)
(50000, 1)
Shape of test data:
(10000, 32, 32, 3)
(10000, 1)


In [17]:
# Transform label indices to one-hot encoded vectors

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)

X_train = np.reshape(X_train,(50000,3072))
X_test = np.reshape(X_test,(10000,3072))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalization of pixel values (to [0-1] range)

X_train /= 255
X_test /= 255

In [18]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=3072))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
sgd = SGD(learning_rate =0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Training

In [19]:
history = model.fit(X_train,y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.2923 - loss: 1.9366 - val_accuracy: 0.3840 - val_loss: 1.7249
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4010 - loss: 1.6748 - val_accuracy: 0.3986 - val_loss: 1.6977
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4261 - loss: 1.5939 - val_accuracy: 0.4071 - val_loss: 1.6544
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4524 - loss: 1.5355 - val_accuracy: 0.4474 - val_loss: 1.5778
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4558 - loss: 1.5151 - val_accuracy: 0.4505 - val_loss: 1.5364
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4712 - loss: 1.4758 - val_accuracy: 0.4578 - val_loss: 1.5356
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4743 - loss: 1.4577 - val_accuracy: 0.4601 - val_loss: 1.5300
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4914 - loss: 1.4168 - 

Evaluating the MLP

In [20]:
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)

In [21]:
print(model.metrics_names)
print(score)

['loss', 'compile_metrics']
[1.5108673572540283, 0.46369999647140503]


## Pytorch MLP text classification IMDB Movie reviews

In [ ]:
!pip install torchtext

  Using cached torchtext-0.18.0-cp310-cp310-manylinux1_x86_64.whl.metadata (7.9 kB)
Using cached torchtext-0.18.0-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset


OSError: /usr/local/lib/python3.10/dist-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [ ]:
# Charger le dataset IMDB
dataset = load_dataset("imdb")

## Keras MLP text classification IMDB Movie reviews

In [22]:
import keras
import tensorflow as tf


In [23]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [25]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Prepare the data

In [26]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Build the model

In [27]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['acc'])

Create a validation set

In [29]:
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

Train the model

In [30]:
history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 269ms/step - acc: 0.5039 - loss: 0.6931 - val_acc: 0.5159 - val_loss: 0.6930
Epoch 2/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 204ms/step - acc: 0.5046 - loss: 0.6931 - val_acc: 0.5080 - val_loss: 0.6930
Epoch 3/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 205ms/step - acc: 0.5004 - loss: 0.6930 - val_acc: 0.4999 - val_loss: 0.6930
Epoch 4/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 205ms/step - acc: 0.4925 - loss: 0.6931 - val_acc: 0.5006 - val_loss: 0.6930
Epoch 5/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - acc: 0.4944 - loss: 0.6931 - val_acc: 0.4995 - val_loss: 0.6930
Epoch 6/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 203ms/step - acc: 0.5067 - loss: 0.6930 - val_acc: 0.4989 - val_loss: 0.6930
Epoch 7/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 203ms/step - acc: 0.5029 - loss: 0.6930 - val_acc: 0.5004 - val_loss: 0.6930
Epoch 8/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - acc: 0.5043 - loss: 0.6929 - val_acc: 0.5034 - val_loss: 0.6930
Epoch 9/40
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step

Evaluate the model

In [31]:
results = model.evaluate(test_data, test_labels)

print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.5157 - loss: 0.6928
[0.6929470896720886, 0.5096799731254578]
